<h1>CS4619: Artificial Intelligence II</h1>
<h1>Transformers</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

<h1>Acknowledgements</h1>
<ul>
    <li>A few of the examples owe a debt to Chollet's book (2nd edition). The table also comes from Chollet's book.
    </li>
</ul>

<h1>Introduction</h1>
<ul>
    <li>We have learned a little about recurrent neural networks (RNNs) and their applications in natural language processing (NLP).</li>
    <li>There have been many improvements to RNNs, such as beam search, bidirectional RNNs and attention mechanisms.</li>
    <li>But a new architecture has emerged that outperforms RNNs: the <b>transformer</b> architecture.
        <ul>
            <li>Ashish Vaswani et al: <i>Attention is all you need.</i> In Proceedings of the 31st International Conference on Neural Information Processing Systems, 2017. (<a href="https://arxiv.org/abs/1706.03762">https://arxiv.org/abs/1706.03762</a>) 
        </ul>
    </li>
    <li>Transformers use <b>self-attention</b> (see below).</li>
    <li>They do not contain any mechanisms for processing input sequentially: no recurrent layers, no 1D-convolutional layers, &hellip; They receive the input sequence all in one go.</li>
    <li>Many of the language models that we reviewed at the end of the previous lecture do in fact use the transformer architecture.</li>
</ul>

<h1>Self-Attention</h1>
<ul>
    <li>The idea of self-attention is to model the dependencies between elements of the input sequence.
        <ul>
            <li>Indeed, the self-attention mechanim enables a transformer to capture <em>long-range</em> dependencies between elements of the input sequence.</li>
            <li>Recall that this was something that caused difficulties for RNNs. Even using LSTMs and GRUs, which were developed to help overcome these difficulties, the problem still remained. 
        </ul>
    </li>
    <li>Consider a word such as ``date''. We have a single word embedding for this word: a vector of numbers. But this word is ambiguous. Its meaning is context-specific: other words that surround it affect its meaning. Consider:
        <ul>
            <li>``I marked the date of the seminar in my calendar.''</li>
            <li>``I enjoyed our date together.''</li>
            <li>``The date seller was not at the market today.''</li>
        </ul>
    </li>
    <li>The idea in self-attention (roughly speaking) is to modify the word embedding based on the word embeddings of the surrounding words.</li>
</ul>

<h2>A Simple Form of Self-Attention</h2>
<ul>
    <li>We start with a simple form of self-attention. Then later we make it more complicated.</li>
    <li>Assume we have an input sentence and we compute the word embedding for each word in that input sentence: $\v{x}_{(1)}, \v{x}_{(2)}, \ldots, \v{x}_{(\mathit{max\_length})}$. Below, for simplicity, I sometimes say ``word'' when I mean ``word embedding''.
    <li>In overview, we do the following for each word $\v{x}_{(i)}$ in the input sequence:
        <ul>
            <li>Compute attention weights $\alpha_{ij}$: simply, each weight is the similarity (dot product) between word $\v{x}_{(i)}$ and every word $\v{x}_{(j)}$ for $j=1\ldots\mathit{max\_length}$ in the input sequence.
                $$\alpha_{ij} = \v{x}_{(i)}\v{x}_{(j)}$$
                (So, if you like, $\alpha_{ij}$ is the relevance of word $j$ to word $i$.)<br />
                Note that these 'weights' are calculated; they are not learned. 
            </li>
            <li>Normalise the weights: for this, we use softmax.</li>
            <li>Compute a context-aware embedding $\v{z}_{(i)}$ for $\v{x}_{(i)}$: the weighted sum of all the word embeddings in the input sequence.
                $$\v{z}_{(i)} = \sum_{j=1}^{\mathit{max\_length}} \alpha_{ij}\v{x}_{(j)}$$
            </li>
        </ul>
    </li>
</ul>

<h3>Example</h3>
<ul>
    <li>Suppose the input sequence is ``the train left the station on time''.</li>
    <li>We get the word embeddings for each word.</li>
    <li>Suppose the current word is word 5: ``station''.</li>
    <li>We compute the similarities (dot products) between the word embedding for ``station'' and the word embeddings for all the words in the input sentence. Suppose this gives the following:
        $$\cv{0.2\\0.8\\0.6\\0.2\\1.0\\0.2\\0.3}$$
        So the similarity between ``train'' and ``station'', for example, is 0.8. Why do we have a 1.0 in there?
    </li>
    <li>Then we normalize using softmax to obtain (roughly):
        $$\cv{0.10\\0.19\\0.15\\0.10\\0.23\\0.10\\0.11}$$
    </li>
    <li>Finally, the context-aware embedding for ``station'' is the weighted sum of the word embeddings:
        $$\v{z}_{(5)} = \mathit{sum}\left(\cv{
        0.10 \times \v{x}_{(1)}\\
        0.19 \times \v{x}_{(2)}\\
        0.15 \times \v{x}_{(3)}\\
        0.10 \times \v{x}_{(4)}\\
        0.23 \times \v{x}_{(5)}\\
        0.10 \times \v{x}_{(6)}\\
        0.11 \times \v{x}_{(7)}}\right)$$
    </li>
    <li>You can see that now we have an embedding for ``station'' that has been modified by its context, i.e. by the other words in the sentence.</li>
</ul>

<h3>Vectorization</h3>
<ul>
    <li>Above, we saw the calculations for one of the words (``station'').</li>
    <li>But, we don't need to do this one word at a time.</li>
    <li>We can vectorize:
        <ul>
            <li>Put the word embeddings into a matrix $\v{X}$. E.g. if there are 7 words and the word embedding dimension is 100, then $\v{X}$ is a $7 \times 100$ matrix.
            <li>Compute the similarities $\v{A}$ between all pairs of words by multiplying $\v{X}$ with itself (acutally with its transpose). In the example, this would give a $7 \times 7$ matrix.</li>
            <li>Normalize each column of the similarity matrix $\v{A}$ using softmax.</li>
            <li>Compute the context-aware vectors for all the words in the input sentence using a matrix multiplication between the normalized similarity matrix and a matrix of the word embeddings:
                $$\v{Z} = \v{A}\v{X}$$
            </li>
        </ul>
    </li>
</ul>

<img src="images/self_attention1.png" />

<h2>A More Advanced Form of Self-Attention</h2>
<ul>
    <li>The simple form of self-attention that we studied above does not have any learnable parameters.</li>
    <li>We introduce three additional weight matrices, $\v{U}_q$, $\v{U}_k$ and $\v{U}_v$.
        <ul>
            <li>They are initialized at random, and then modified during back-prop.<br />
                Note that these weights are learned &mdash; unlike the ones in $\v{A}$.</li>
            <li>Let the embedding dimension of the word embeddings be $d$. Then, it is common for $\v{U}_q$, $\v{U}_k$ and $\v{U}_v$ to be $d \times d$ matrices. And this is what we will assume in our presentation here.</li>
            <li>(Advanced, ignore: more generally, we can choose values $d_k$ and $d_v$ and then $\v{U}_q$ and $\v{U}_k$ can be $d_k \times d$, and $\v{U}_v$ can be $d_v \times d$.)</li>
        </ul>
    </li>
    <li>How do we use these weight matrices?
        <ul>
            <li>We multiply the word embeddings $\v{x}_{(i)}$ by each of the matrices:
                $$\v{q}_{(i)} = \v{U}_q\v{x}_{(i)}$$
                $$\v{k}_{(i)} = \v{U}_k\v{x}_{(i)}$$
                $$\v{v}_{(i)} = \v{U}_v\v{x}_{(i)}$$
            </li>
            <li>Previously, we computed the weights as the similarity (dot product) of the word embeddings: $\alpha_{ij} = \v{x}_{(i)}\v{x}_{(j)}$. But, now:
                $$\alpha_{ij} = \v{q}_{(i)}\v{k}_{(j)}$$
            </li>
            <li>Then, we normalise the weights, as before: softmax. (Advanced, ignore: In fact, in transformers, we scale the weights by multiplying them by $1/\sqrt{d}$ before we take the softmax. This gives a 'smoother' softmax distrbution.)</li>
            <li>Finally, we compute a context-aware embedding. Previously, we computed a weighted sum: $\v{z}_{(i)} = \sum_{j=1}^{\mathit{max\_length}} \alpha_{ij}\v{x}_{(j)}$. But, now:
              $$\v{z}_{(i)} = \sum_{j=1}^{\mathit{max\_length}} \alpha_{ij}\v{v}_{(j)}$$
            </li>
        </ul>
    </li>
</ul>

<img src="images/self_attention2.png" />

<h3>Query, key and value</h3>
<ul>
    <li>For each word, we computed three vectors, $\v{q}_{(i)}$, $\v{k}_{(i)}$ and $\v{v}_{(i)}$.</li>
    <li>People refer to these three as the <b>query</b>, <b>key</b> and <b>value</b> vectors.</li>
    <li>Why? The inventors of transformers needed three names. They hijacked three names from the field of non-relational databases. In a non-relational database, we enter a query, it is matched against a key and a value is retrieved. If we are being charitable, we can see that in transformers query vectors are matched with key vectors when we compute similarities, and value vectors are part of the final result. So there is a tenuous connection.</li>
     <li>For me, these names are unhelpful. They might just as well have called them Huey, Dewey and Louie in honour of Donald Duck's nephews!
    </li>
    <li>(Adavanced, ignore: We used $\v{x}_{(i)}$ for calculating the query, key and value. This is what we do for sequence classification, e.g. for sentiment analysis. But, in fact, you could use a different vector for each. For example, in machine translation between English and French, a training example will have both an English sentence and a French sentence. You might use French words when computing the query, and English words when computing the keys and values.)</li>
</ul>

<h2>Multi-Head Attention</h2>
<ul>
    <li>Let's refer to the three matrices $\v{U}_q$, $\v{U}_k$ and $\v{U}_v$ as a <b>head</b>. Then, what we have described is a transformer that has a single head.</li>
    <li>But transformers use multi-head attention. Guess what: we have mutliple heads. In other words, if we have $h$ heads then we have $h$ sets of these matrices:
        <ul>
            <li>$\v{U}_{q_1}$, $\v{U}_{k_1}$, $\v{U}_{v_1}$</li>
            <li>$\v{U}_{q_2}$, $\v{U}_{k_2}$, $\v{U}_{v_2}$</li>
            <li>&hellip;</li>
            <li>$\v{U}_{q_h}$, $\v{U}_{k_h}$, $\v{U}_{v_h}$</li>
        </ul>
    </li>
    <li>Now, instead of computing one query vector, key vector and value vector per word, we compute $h$ of them.</li>
    <li>Why do we want multiple heads? It gives the transformer more parameters and an opportunity to learn $h$ different representations. It's a bit like why we have multiple feature maps in a convolutional layer.
        <figure style="text-align: center;">
            <img src="images/attention_examples.png" />
            <figcaption>Evidence that different heads learn to perform different tasks.<br />
                Taken from the <i>Attention is all you need</i> paper)
            </figcaption>
        </figure>
    </li>
    <li>(Advanced, ignore: In fact, instead of having $h$ sets of matrices, we combine them into just 3 matrices and use Boolean masks to access regions of the matrices. This improves efficiency: it means we can use just one matrix multiplication instead of $h$ matrix multiplications.)</li>
    <li>The outputs of the multiple heads are concatenated.</li>
</ul>

<figure style="text-align: center;">
    <img src="images/multi_head_attention.png" />
    <figcaption>Taken from the <i>Attention is all you need</i> paper<br />
        (Linear = Dense layers)
    </figcaption>
</figure>

<h2>Positional Encoding</h2>
<h3>Sets, not sequences</h3>
<ul>
    <li>Self-attention is a <em>set</em>-processing mechanism, not a <em>sequence</em>-processing mechanism.
        <ul>
            <li>We did not feed in an input sentence $\v{x}_{(1)}, \v{x}_{(2)}, \ldots, \v{x}_{(\mathit{max\_length})}$ one word at a time.</li>
            <li>We treated the input sentence as a matrix, where each row of the matrix is one element of the input sequence.
                And the whole matrix is fed in as a single input in one go.
            </li>
            <li>This is great for parallelization (although we should acknowledge that transformers are quite expensive to train, even when parallelized).</li>
            <li>But it means that the input is now a set (a set of rows), not a sequence!
                <ul>
                    <li>You could shuffle the rows and it will make no difference: you will calculate the same pairwise similarities and the same context-aware representations.
                </ul>
            </li>
        </ul>
    </li>
    <li>We don't want this! We have lost word ordering. It makes ``Dogs hate cats'' no different from ``Cats hate dogs'' &mdash; both are sets (matrices) containing the word embeddings for ``cats'', ``dogs'' and ``hates''.
    </li>
    <li>Transformers need an extra mechanism to re-inject word ordering information.
        <table>
            <tr><th></th><th>Word order awareness</th><th>Context-awareness</th></tr>
            <tr><th>Bag-of-words (unigrams)</th><td>No</td><td>No</td></tr>
            <tr><th>Bag-of-words (bigrams)</th><td>Very limited</td><td>No</td></tr>
            <tr><th>RNN</th><td>Yes</td><td>No</td></tr>
            <tr><th>Self-attention</th><td>No</td><td>Yes</td></tr>
            <tr><th>Transformer</th><td>Yes</td><td>Yes</td></tr>
        </table>
    </li>
</ul>

<h3>Re-injecting word order information</h3>
<ul>
    <li>To re-inject word order information, we will include the word's position in the input sentence to each word embedding. Let's consider three ways of doing this.</li>
    <li>Simple: Concatenate the word position to the word embedding. 
        <ul>
            <li>Concatenate 0 to the word embedding of the first word.</li>
            <li>Concatenate 1 to the word embedding of the second word.</li>
            <li>And so on.</li>
        </ul>
        This is not ideal: these integers would dominate the values in the word embeddings, making the word embeddings themselves negligble.
    </li>
    <li>The original transformer: Add the sine and/or cosine of the position.
        <ul>
            <li>This means you add values in the range $[-1, 1]$</li>
            <li>The values vary cyclically.</li>
        </ul>
    </li>
    <li>Positional embedding: learn an embedding of the position.
        <ul>
            <li>We know that an embedding layer maps from a word index (i.e. its index in the vocabulary) to a $d$-dimensional vector.</li>
            <li>So why not use another embedding layer to map from word positions (i.e. where it is in the sentence) to another $d$-dimensional vector.</li>
            <li>Then add the word embedding and the positional embedding.</li>
        </ul>
    </li>
</ul>

<h2>Transformers</h2>
<ul>
    <li>Transformers put all these ideas together with others. Details unimportant in CS4619.
        <ul>
            <li>They add some dense layers.</li>
            <li>They add some skip links (see the final lecture of AI1 to remind yourself what these are).</li>
            <li>They add layer normalization. Layer normalization is similar to, but different from batch normalization. Batch normalization computes means and standard deviatons from all examples in a mini-batch, but this does not work well for sequence data. Instead, layer normalization normalizes each input sequence independently from the others. In other words, it computes means and standard deviations from all inputs in each sequence.</li>
        </ul>
    </li>
    <li>Here is the diagram from the original paper:
        <figure style="text-align: center;">
            <img src="images/transformer_encoder.png" />
        </figure>
        They used 8 heads and a stack of 6 of these layers. 
    </li>
    <li>In fact, what we have discussed so far (and the diagram) is just the transformer encoder. You'll then want more layers to do your task.
        <ul>
            <li>E.g. for sentiment analysis (negative, neural, positive), you want a final dense layer containing three neurons as your output layer.</li>
            <li>E.g. for a word-level language model, you want a final dense layer containing one neuron per word in your vocabulary as your output layer.</li>
        </ul>
    </li>
    <li>If you want to do sequence-to-sequence tasks, such as Machine Translation, code completion or question-answering, then you will want to connect your transformer encoder to a transformer decoder.
    </li>
</ul>

<h2>Training and Pretraining</h2>
<ul>
    <li>Your chances of training a transformer from scratch are small.</li>
    <li>Instead, they are trained by resource-rich organizations on large datasets for next-word prediction (self-supervised learning).</li>
    <li>Then, these pretrained transformers are reused and fine-tuned for more specific tasks using a labeled dataset.</li>
    <li>Famous examples of popular large-scale language models:
        <ul>
            <li>GPT: Generative Pretrained Transformer, developed by OpenAI
                <ul>
                    <li>GPT has gone through several versions, GPT-1, GPT-2, GPT-3</li>
                    <li>One difference is expectations about fine-tuning. It is claimed that, to varying degrees, they do not need fine-tuning (lookup discussions of zero-shot, one-shot or few-shot task transfer if you're interested).</li>
                    <li>A final difference is that GPT-3 uses sparse attention, which means that attention weights are not computed for all pairs, just for a subset.</li>
                </ul>
            </li>
            <li>BERT: Bidirectional Encode Representations from Transformers, developed by Google
                <ul>
                    <li>GPT uses self-attention but only from preceding words, whereas BERT uses self-attenton from preceding and succeeding words.</li>
                    <li>Instead of next-word prediction, BERT is pretrained on predicting masked words, which might occur anywhere in the input.</li>
                </ul>
            </li>
            <li>BART: Bidirectional and Auto-Regressive Transformers, developed by Facebook/Meta
                <ul>
                    <li>GPT's speciality is generating text; BERT's is classifying text. BART generalises them to accomplish both tasks.</li>
                </ul>
            </li>
        </ul>
        &hellip; and dozens more!
    </li>
</ul>

<h2>Hugging Face</h2>
<ul>
    <li>Hugging Face is an AI company: <a href="https://huggingface.co/">https://huggingface.co/</a></li>
    <li>They make pretrained models available via their <code>transformers</code> library:
        <a href="https://huggingface.co/transformers/v3.3.1/pretrained_models.html">https://huggingface.co/transformers/v3.3.1/pretrained_models.html</a>
    </li>
    <li>See their installation instructions, but at the time of writing the instructions are:
        <ol>
            <li>Activate tyour virtual environment.</li>
            <li>Install: <code>pip install transformers</code></li>
            <li>Check: <code>python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"</code><br />
                It should print: <code>[{'label': 'POSITIVE', 'score': 0.9998704791069031}]</code></li>
        </ol>
    </li>
</ul>

In [3]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

2022-12-08 15:40:21.752376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
set_seed(123)

generator("Opinions about Derek's AI modules vary from", max_length=40, num_return_sequences=3)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': "Opinions about Derek's AI modules vary from one party to another. There is generally disagreement over whether the human element to Derek's ability to learn AI is due to his ability to recognize others more"},
 {'generated_text': "Opinions about Derek's AI modules vary from country to country, but the team has decided to focus on the high end and have a minimum of six hours per day devoted to this mission.\n\n"},
 {'generated_text': "Opinions about Derek's AI modules vary from person to person, but the gist of his blog post is that the data provided indicates that Derek has made some sort of major performance improvement over his previous"}]

<h2>Final Remarks</h2>
<ul>
    <li>What about applying transformers to image processing?
        <ul>
            <li>On large datasets, there are some promising results, especially considering robustness (in other words, performance on unseen out-of-distribution examples). See the ViT (Vision Transformer) architecture: <a href="https://arxiv.org/abs/2010.11929">https://arxiv.org/abs/2010.11929</a></li>
            <li>But this paper suggests that it is other aspects of the transformer architecture (e.g. its more aggressive downsampling) that contributes most. If these ideas are incorporated into ConvNets, then ConvNet robustness can rival Transformer robustness: <a href="https://arxiv.org/abs/2206.03452">https://arxiv.org/abs/2206.03452</a></li>
        </ul>
    </li>
    <li>Meanwhile, researchers are trying transformers wherever they have set-based or sequence-based examples, e.g. recommender systems and reinforcement learning.
    </li>
</ul>